In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("data/raw/adult.csv")
print(df.head())

   age  workclass  fnlwgt     education  educational-num      marital-status  \
0   25    Private  226802          11th                7       Never-married   
1   38    Private   89814       HS-grad                9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm               12  Married-civ-spouse   
3   44    Private  160323  Some-college               10  Married-civ-spouse   
4   18          ?  103497  Some-college               10       Never-married   

          occupation relationship   race  gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black    Male             0             0   
1    Farming-fishing      Husband  White    Male             0             0   
2    Protective-serv      Husband  White    Male             0             0   
3  Machine-op-inspct      Husband  Black    Male          7688             0   
4                  ?    Own-child  White  Female             0             0   

   hours-per-week native-country incom

In [12]:
print(df.isnull().sum())

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64


In [13]:
print(df.isnull().mean() * 100)

age                0.0
workclass          0.0
fnlwgt             0.0
education          0.0
educational-num    0.0
marital-status     0.0
occupation         0.0
relationship       0.0
race               0.0
gender             0.0
capital-gain       0.0
capital-loss       0.0
hours-per-week     0.0
native-country     0.0
income             0.0
dtype: float64


In [14]:
print(df[df.isnull().any(axis=1)].head())

Empty DataFrame
Columns: [age, workclass, fnlwgt, education, educational-num, marital-status, occupation, relationship, race, gender, capital-gain, capital-loss, hours-per-week, native-country, income]
Index: []


In [15]:
df_sem_nulos = df.dropna()

df_menos_colunas = df.dropna(axis=1, thresh=0.5*len(df))

In [16]:
df_preenchido = df.fillna(df.median(numeric_only=True))

In [17]:
for col in df.select_dtypes(include=["object"]).columns:
    df_preenchido[col] = df_preenchido[col].fillna(df[col].mode()[0])


In [18]:
df_preenchido.to_csv("data/processed/adult_clean.csv", index=False)

In [19]:
def detectar_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return col[(col < limite_inferior) | (col > limite_superior)]

outliers = {}
for col in df_preenchido.select_dtypes(include=[np.number]).columns:
    outliers[col] = detectar_outliers(df_preenchido[col])

print("\nQuantidade de outliers detectados por coluna:")
for col, vals in outliers.items():
    print(f"{col}: {len(vals)}")

df_tratado = df_preenchido.copy()
for col in df_tratado.select_dtypes(include=[np.number]).columns:
    Q1 = df_tratado[col].quantile(0.25)
    Q3 = df_tratado[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    mediana = df_tratado[col].median()
    df_tratado.loc[(df_tratado[col] < limite_inferior) | (df_tratado[col] > limite_superior), col] = mediana

df_tratado.to_csv("data/processed/adult_final.csv", index=False)


Quantidade de outliers detectados por coluna:
age: 216
fnlwgt: 1453
educational-num: 1794
capital-gain: 4035
capital-loss: 2282
hours-per-week: 13496


C:\Users\Erlanny Rodrigues\AppData\Local\Temp\ipykernel_3968\1842969414.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '178144.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_tratado.loc[(df_tratado[col] < limite_inferior) | (df_tratado[col] > limite_superior), col] = mediana
